In [1]:
import cv2
from PIL import Image
import numpy as np
import os
import glob

In [8]:
# image_dirs = [
#     "5_Yb4AMwr0vNE",
#     "12_1mWNahzcsAc",
#     "14_3RVwoJD2px0",
#     "53_kPNQnO6YiHM",
#     "59_NzCdo4wzMqs",
#     "69_bpTLbZ1juqg",
#     "79_M58q5Gxp4f4",
#     "82_xz40b41FHfs",
#     "102_aME6mBuWEAc",
#     "107_tQA8kJXlTwc",
#     "123_gi3DeFY0cfw",
#     "143_4VYUTM1pQBc",
#     "150_OTzCuoWZoYo",
#     "502__dL1I55zC7Q"
# ]
# directories = [
#     "fgr",
#     "mask",
#     "pha_vid_0911_from-seg_refined",
#     "pha_vid_0911_from-seg",
#     "pha_vid_0913_from-seg"
# ]
# data_dir = "/home/chuongh/vm2m/data/VIPSeg/out"

# image_dirs = [
#     "6_production_id_4880454_2160p"
# ]
# image_dirs = [
# "2_pexels_videos_2795168_2160p",
# "2_production_id_4016505_2160p",
# "3_pexels-artem-podrez-6952660_Original",
# "3_production_id_4122569_2160p",
# "2_pexels_videos_2795172_2160p",  
# "2_production_id_4426909_2160p",  
# "3_pexels-cottonbro-5329478_2160p",
# "4_production_id_4005145_2160p"]
image_dirs = [
    "2_pexels-artem-podrez-6003997_2160p",
    "2_pexels-cottonbro-5329614_2160p",
    "2_pexels_videos_2795168_2160p",
    "2_pexels_videos_2795172_2160p",
    "2_production_id_4016505_2160p",
    "2_production_id_4321723_2160p",
    "2_production_id_4426909_2160p",
    "3_pexels-artem-podrez-6952660_Original",
    "3_pexels-cottonbro-5329478_2160p",
    "3_production_id_4122569_2160p",
    "3_production_id_4860318_2160p",
    "4_pexels-artem-podrez-8088627_Original",
    "4_pexels-pavel-danilyuk-8058107_Original",
    "4_production_id_4005145_2160p",
    "5_video_2160p",
    "5_video_2160p_1",
    # "6_production_id_4880454_2160p",
    # "6_production_id_4880458_2160p"
]
directories = [
    "fgr",
    "xmem",
    "sparsemat_1025/alpha_pred",
    "mgm-tcvom_1025/alpha_pred",
    "vhm_sparsity_1025_temp/alpha_pred"
]
data_dir = "/mnt/localssd/syn/benchmark/real_qual_filtered"

In [9]:
for video_name in image_dirs:
    n_i = len(glob.glob(os.path.join(data_dir, "mask", video_name,"*/*.png")))
    n_f = len(os.listdir(os.path.join(data_dir, "fgr", video_name)))
    n_i = n_i // n_f
    !rm -rf {os.path.join(data_dir, "temp")}
    os.makedirs(os.path.join(data_dir, "temp"), exist_ok=True)
    
    for inst_i in range(n_i):
        prev_pred = []
        # try:
        all_names = sorted(os.listdir(os.path.join(data_dir, "fgr", video_name)))
        for image_name in all_names:
            image_name = image_name.replace(".jpg", "")
            all_paths = []
            for dirname in directories:
                if dirname == "fgr":
                    all_paths.append(os.path.join(data_dir, dirname, video_name, f"{image_name}.jpg"))
                else:
                    all_paths.append(os.path.join(data_dir, dirname, video_name, image_name, "{:02d}.png".format(inst_i)))
            all_images = []
            for path in all_paths:
                all_images.append(np.array(Image.open(path).convert("RGB")))
            
            # Combine to single image
            # diff = np.zeros_like(all_images[0])
            diff = cv2.absdiff(all_images[3], all_images[4]) # > 0).astype(np.uint8) * 255
            all_images = all_images[:2] + [diff] + all_images[2:]
            row1 = np.concatenate(all_images[:3], axis=1)
            row2 = np.concatenate(all_images[3:], axis=1)
            diff_notemp = (cv2.absdiff(all_images[3], prev_pred[0]) > 0).astype('uint8') * 255 if len(prev_pred) > 0 else np.zeros_like(all_images[3])
            diff_temp = (cv2.absdiff(all_images[4], prev_pred[1]) > 0).astype('uint8') * 255 if len(prev_pred) > 0 else np.zeros_like(all_images[4])
            diff_temp_bi = (cv2.absdiff(all_images[5], prev_pred[2]) > 0).astype('uint8') * 255 if len(prev_pred) > 0 else np.zeros_like(all_images[5])
            row3 = np.concatenate([diff_notemp, diff_temp, diff_temp_bi], axis=1)

            combined = np.concatenate([row1, row2, row3], axis=0)
            prev_pred = all_images[3:6]
            ratio = 2500.0 / combined.shape[1]
            combined = cv2.resize(combined, (0, 0), fx=ratio, fy=ratio)
            
            cv2.imwrite(os.path.join(data_dir, "temp", f"{image_name}.jpg"), combined[:, :, ::-1])
        out_vid = f"{data_dir}/1011_{video_name}_{inst_i}.mp4"
        in_pattern = f"{data_dir}/temp/*.jpg"
        !ffmpeg -y -framerate 3 -pattern_type glob -i "{in_pattern}" -c:v libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p {out_vid}
        # except:
        #     pass

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


In [3]:
directories = [
    "images",
    "alphas",
    "masks",
    "mgm_single_tcvom",
    "mgm_stacked_tcvom",
    "ours"
]
data_dir = "/home/chuongh/vm2m/data/vis/real"
image_dirs = os.listdir(os.path.join(data_dir, "images"))

In [4]:
def log_diff_cmap(pred, gt):
    pred = pred[:, :, 0]
    gt = gt[:, :, 0]
    diff = np.abs(pred - gt)
    diff = np.log(diff + 1e-6)
    diff = diff / np.log(255)
    diff = (diff * 255).astype('uint8')
    diff = cv2.applyColorMap(diff, cv2.COLORMAP_JET)
    return diff

In [7]:
for video_name in image_dirs:
    n_i = len(glob.glob(os.path.join(data_dir, "masks", video_name,"*/*.png")))
    n_f = len(os.listdir(os.path.join(data_dir, "images", video_name)))
    n_i = n_i // n_f
    !rm -rf {os.path.join(data_dir, "temp")}
    os.makedirs(os.path.join(data_dir, "temp"), exist_ok=True)
    
    for inst_i in range(n_i):
        prev_pred = []
        # try:
        all_names = sorted(os.listdir(os.path.join(data_dir, "images", video_name)))
        for image_name in all_names:
            image_name = image_name.replace(".jpg", "")
            all_paths = []
            for dirname in directories:
                if dirname == "images":
                    all_paths.append(os.path.join(data_dir, dirname, video_name, f"{image_name}.jpg"))
                else:
                    mask_path = os.path.join(data_dir, dirname, video_name, image_name, "{:02d}.png".format(inst_i))
                    if os.path.exists(mask_path):
                        all_paths.append(mask_path)
                    else:
                        all_paths.append(os.path.join(data_dir, dirname, video_name, image_name + ".png"))
            all_images = []
            for path in all_paths:
                all_images.append(np.array(Image.open(path).convert("RGB")))
            
            # Combine to single image
            row1 = np.concatenate(all_images[:3], axis=1)
            row2 = np.concatenate(all_images[3:], axis=1)
            diff1 = log_diff_cmap(all_images[1], all_images[3])
            diff2 = log_diff_cmap(all_images[1], all_images[4])
            diff3 = log_diff_cmap(all_images[1], all_images[5])
            row3 = np.concatenate([diff1, diff2, diff3], axis=1)

            combined = np.concatenate([row1, row2, row3], axis=0)
            prev_pred = all_images[3:6]
            ratio = 2500.0 / combined.shape[1]
            combined = cv2.resize(combined, (0, 0), fx=ratio, fy=ratio)
            
            cv2.imwrite(os.path.join(data_dir, "temp", f"{image_name}.jpg"), combined[:, :, ::-1])
        out_vid = f"{data_dir}/1011_{video_name}_{inst_i}.mp4"
        in_pattern = f"{data_dir}/temp/*.jpg"
        !ffmpeg -y -framerate 3 -pattern_type glob -i "{in_pattern}" -c:v libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p {out_vid}
        # except:
        #     pass

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


In [8]:
image_dirs

['Pexels_Videos_2795385',
 'production_ID_4265392',
 'production_ID_4170843',
 'production_ID_5197613',
 '3048929',
 'production_ID_4098957',
 'production_ID_4531303',
 'production_ID_4918184',
 'production_ID_4065480',
 'production_ID_4728157',
 'production_ID_4099014',
 'pexels-artem-podrez-5457692',
 'production_ID_4736798',
 'production_ID_4594921',
 'production_ID_4625474',
 'pexels-rodnae-productions-5642515',
 'production_ID_4825047',
 'production_ID_5158552']

Bad pipe message: %s [b"j'\xec\x9dqAG\x81\x0c\r1\x9e\x1c\x8b\x14\xc7E\xd4 \xab\x8b\x0e\xa2:\x19$\xbb\xb4G\xd6\xe3\x7f\x00\x94Z\xaa\xa9\x13\xc9\x03\x072MMa\xd1\x06o\xc5C;\x00>\x13\x02\x13\x03\x13\x01\xc0,\xc00\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0+\xc0/\x00\x9e\xc0$\xc0(\x00k\xc0#\xc0'\x00g\xc0\n\xc0\x14\x009\xc0\t\xc0\x13\x003\x00\x9d\x00\x9c\x00=\x00<\x005\x00/\x00\xff\x01\x00\x01u\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x03\x03\x02\x03\x03\x01\x02\x01\x03\x02"]
Bad pipe message: %s [b'\x04']
Bad pipe message: %s [b'\x02', b'\x00+\x00\t\x08']
Bad pipe message: %s [b'\x03\x03']
Bad pipe message: %s [b'\x03\x01']


For the qual set

In [12]:
directories = [
    "images",
    "masks",
    "mgm_single_tcvom",
    "mgm_stacked_tcvom",
    "ours"
]
data_dir = "/home/chuongh/vm2m/data/vis/real_qual"
image_dirs = sorted(os.listdir(os.path.join(data_dir, "images")))

In [13]:
for video_name in image_dirs:
    n_i = len(glob.glob(os.path.join(data_dir, "masks", video_name,"*/*.png")))
    n_f = len(os.listdir(os.path.join(data_dir, "images", video_name)))
    n_i = n_i // n_f
    !rm -rf {os.path.join(data_dir, "temp")}
    os.makedirs(os.path.join(data_dir, "temp"), exist_ok=True)
    
    for inst_i in range(n_i):
        prev_pred = []
        # try:
        all_names = sorted(os.listdir(os.path.join(data_dir, "images", video_name)))
        for image_name in all_names:
            image_name = image_name.replace(".jpg", "")
            all_paths = []
            for dirname in directories:
                if dirname == "images":
                    all_paths.append(os.path.join(data_dir, dirname, video_name, f"{image_name}.jpg"))
                else:
                    mask_path = os.path.join(data_dir, dirname, video_name, image_name, "{:02d}.png".format(inst_i))
                    if os.path.exists(mask_path):
                        all_paths.append(mask_path)
                    else:
                        all_paths.append(os.path.join(data_dir, dirname, video_name, image_name + ".png"))
            all_images = []
            for path in all_paths:
                all_images.append(np.array(Image.open(path).convert("RGB")))
            
            # Combine to single image
            all_images = all_images[:2] + [cv2.absdiff(all_images[1], all_images[4])] + all_images[2:]
            row1 = np.concatenate(all_images[:3], axis=1)
            row2 = np.concatenate(all_images[3:], axis=1)
            diff1 = log_diff_cmap(prev_pred[0], all_images[3]) if len(prev_pred) > 0 else np.zeros_like(all_images[3])
            diff2 = log_diff_cmap(prev_pred[1], all_images[4]) if len(prev_pred) > 0 else np.zeros_like(all_images[4])
            diff3 = log_diff_cmap(prev_pred[2], all_images[5]) if len(prev_pred) > 0 else np.zeros_like(all_images[5])
            row3 = np.concatenate([diff1, diff2, diff3], axis=1)

            combined = np.concatenate([row1, row2, row3], axis=0)
            prev_pred = all_images[3:6]
            ratio = 2500.0 / combined.shape[1]
            cv2.putText(combined, image_name, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 5)
            combined = cv2.resize(combined, (0, 0), fx=ratio, fy=ratio)
            
            cv2.imwrite(os.path.join(data_dir, "temp", f"{image_name}.jpg"), combined[:, :, ::-1])
            
        out_vid = f"{data_dir}/{video_name}_{inst_i}.mp4"
        in_pattern = f"{data_dir}/temp/*.jpg"
        
        !ffmpeg -y -framerate 3 -pattern_type glob -i "{in_pattern}" -c:v libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p {out_vid}
    

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


In [3]:
directories = [
    "images",
    "masks",
    "instmatt",
    "sparsemat_hr",
    "mgm_single_tcvom",
    "mgm_stacked_tcvom",
    "ours"
]
data_dir = "/home/chuongh/vm2m/data/vis/real_qual"
image_dirs = sorted(os.listdir(os.path.join(data_dir, "images")))

In [13]:
for video_name in image_dirs:
    n_i = len(glob.glob(os.path.join(data_dir, "masks", video_name,"*/*.png")))
    n_f = len(os.listdir(os.path.join(data_dir, "images", video_name)))
    n_i = n_i // n_f
    !rm -rf {os.path.join(data_dir, "temp")}
    os.makedirs(os.path.join(data_dir, "temp"), exist_ok=True)
    
    for inst_i in range(n_i):
        prev_pred = []
        # try:
        all_names = sorted(os.listdir(os.path.join(data_dir, "images", video_name)))
        for image_name in all_names:
            image_name = image_name.replace(".jpg", "")
            all_paths = []
            for dirname in directories:
                if dirname == "images":
                    all_paths.append(os.path.join(data_dir, dirname, video_name, f"{image_name}.jpg"))
                else:
                    mask_path = os.path.join(data_dir, dirname, video_name, image_name, "{:02d}.png".format(inst_i))
                    if os.path.exists(mask_path):
                        all_paths.append(mask_path)
                    else:
                        all_paths.append(os.path.join(data_dir, dirname, video_name, image_name + ".png"))
            all_images = []
            for path in all_paths:
                all_images.append(np.array(Image.open(path).convert("RGB")))
            
            # Combine to single image
            all_images = all_images[:2] + [np.zeros_like(all_images[0]) for _ in range(2)] + [cv2.absdiff(all_images[1], all_images[6])] + all_images[2:]
            row1 = np.concatenate(all_images[:5], axis=1)
            row2 = np.concatenate(all_images[5:], axis=1)
            diff1 = log_diff_cmap(prev_pred[0], all_images[5]) if len(prev_pred) > 0 else np.zeros_like(all_images[3])
            diff2 = log_diff_cmap(prev_pred[1], all_images[6]) if len(prev_pred) > 0 else np.zeros_like(all_images[4])
            diff3 = log_diff_cmap(prev_pred[2], all_images[7]) if len(prev_pred) > 0 else np.zeros_like(all_images[5])
            diff4 = log_diff_cmap(prev_pred[3], all_images[8]) if len(prev_pred) > 0 else np.zeros_like(all_images[6])
            diff5 = log_diff_cmap(prev_pred[4], all_images[9]) if len(prev_pred) > 0 else np.zeros_like(all_images[7])
            row3 = np.concatenate([diff1, diff2, diff3, diff4, diff5], axis=1)

            combined = np.concatenate([row1, row2, row3], axis=0)
            prev_pred = all_images[5:10]
            ratio = 2500.0 / combined.shape[1]
            cv2.putText(combined, image_name, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 5)
            combined = cv2.resize(combined, (0, 0), fx=ratio, fy=ratio)
            
            cv2.imwrite(os.path.join(data_dir, "temp", f"{image_name}.jpg"), combined[:, :, ::-1])
            
        out_vid = f"{data_dir}/{video_name}_{inst_i}.mp4"
        in_pattern = f"{data_dir}/temp/*.jpg"
        
        !ffmpeg -y -framerate 3 -pattern_type glob -i "{in_pattern}" -c:v libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p {out_vid}
    

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


Bad pipe message: %s [b"\xee\xd2\xbe\xdd\xee)\xb8\xd08O\x80\x8b\xb71\xb8\x0fn\x84 \x19aB\xf4\x18H\xa1H&4\x9e\xda_]MM:W\x8b\x82n\xac\xcf\xae3\xc0L\xb0\x86\xa7\x1e\x0f\x00>\x13\x02\x13\x03\x13\x01\xc0,\xc00\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0+\xc0/\x00\x9e\xc0$\xc0(\x00k\xc0#\xc0'\x00g\xc0\n\xc0\x14\x009\xc0\t\xc0\x13\x003\x00\x9d\x00"]
Bad pipe message: %s [b'=\x00<\x005\x00/\x00\xff\x01\x00\x01u\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x03\x03\x02\x03\x03\x01\x02\x01\x03\x02\x02\x02\x04\x02\x05\x02\x06\x02\x00+\x00\t\x08\x03\x04\x03\x03\x03\x02\x03\x01\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xe0\xfd,q\x89\xbbb\xf7~%$\xcf\xea;9?e\x03G\xce\xd3\xcb\xd3\r\xe8']


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


In [12]:
len(prev_pred)

4

In [16]:
directories = [
    "images",
    "masks",
    "instmatt",
    "ours"
]
data_dir = "/home/chuongh/vm2m/data/vis/teaser_2"
image_dirs = sorted(os.listdir(os.path.join(data_dir, "images")))

In [17]:
for video_name in image_dirs[::-1]:
    n_i = len(glob.glob(os.path.join(data_dir, "masks", video_name,"*/*.png")))
    n_f = len(os.listdir(os.path.join(data_dir, "images", video_name)))
    n_i = n_i // n_f
    !rm -rf {os.path.join(data_dir, "temp")}
    os.makedirs(os.path.join(data_dir, "temp"), exist_ok=True)
    
    
    prev_pred = []
    # try:
    all_names = sorted(os.listdir(os.path.join(data_dir, "images", video_name)))
    for image_name in all_names:
        image_name = image_name.replace(".jpg", "")
        all_paths = []
        for dirname in directories:
            if dirname == "images":
                all_paths.append(os.path.join(data_dir, dirname, video_name, f"{image_name}.jpg"))
            else:
                for inst_i in range(n_i):
                    mask_path = os.path.join(data_dir, dirname, video_name, image_name, "{:02d}.png".format(inst_i))
                    if os.path.exists(mask_path):
                        all_paths.append(mask_path)
                    else:
                        all_paths.append(os.path.join(data_dir, dirname, video_name, image_name + ".png"))
        all_images = []
        for path in all_paths:
            all_images.append(np.array(Image.open(path).convert("RGB")))
        
        # Combine to single image
        row1 = np.concatenate(all_images[:3], axis=1)
        # row2 = np.concatenate(all_images[3:5], axis=1)

        # compose image with green background
        green_bg = np.zeros_like(all_images[0])
        green_bg[:, :, 1] = 255
        alpha = all_images[3] / 255.0
        image1 = all_images[0] * alpha + (1.0 - alpha) * green_bg
        alpha = all_images[4] / 255.0
        image2 = all_images[0] * alpha + (1.0 - alpha) * green_bg
        row2 = np.concatenate([image1, image2, np.zeros_like(green_bg)], axis=1)

        # green_bg = np.zeros_like(all_images[0])
        # green_bg[:, :, 1] = 255
        alpha = all_images[5] / 255.0
        image1 = all_images[0] * alpha + (1.0 - alpha) * green_bg
        alpha = all_images[6] / 255.0
        image2 = all_images[0] * alpha + (1.0 - alpha) * green_bg
        row3 = np.concatenate([image1, image2, np.zeros_like(green_bg)], axis=1)
        # diff1 = log_diff_cmap(prev_pred[0], all_images[5]) if len(prev_pred) > 0 else np.zeros_like(all_images[3])
        # diff2 = log_diff_cmap(prev_pred[1], all_images[6]) if len(prev_pred) > 0 else np.zeros_like(all_images[4])
        # diff3 = log_diff_cmap(prev_pred[2], all_images[7]) if len(prev_pred) > 0 else np.zeros_like(all_images[5])
        # diff4 = log_diff_cmap(prev_pred[3], all_images[8]) if len(prev_pred) > 0 else np.zeros_like(all_images[6])
        # diff5 = log_diff_cmap(prev_pred[4], all_images[9]) if len(prev_pred) > 0 else np.zeros_like(all_images[7])
        # row3 = np.concatenate([diff1, diff2, diff3, diff4, diff5], axis=1)

        combined = np.concatenate([row1, row2, row3], axis=0)
        # prev_pred = all_images[5:10]
        ratio = 2500.0 / combined.shape[1]
        cv2.putText(combined, image_name, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 5)
        combined = cv2.resize(combined, (0, 0), fx=ratio, fy=ratio)
        
        cv2.imwrite(os.path.join(data_dir, "temp", f"{image_name}.jpg"), combined[:, :, ::-1])
        
    out_vid = f"{data_dir}/{video_name}_{inst_i}.mp4"
    in_pattern = f"{data_dir}/temp/*.jpg"
    
    !ffmpeg -y -framerate 3 -pattern_type glob -i "{in_pattern}" -c:v libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p {out_vid}
    

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


In [15]:
directories = [
    "images",
    "masks",
    "ours",
    "ours_ss",
    "ours_tokens"
]
data_dir = "/home/chuongh/vm2m/data/vis/real_qual"
image_dirs = sorted(os.listdir(os.path.join(data_dir, "images")))

In [22]:
os.path.join(data_dir, "images", video_name)

'/home/chuongh/vm2m/data/vis/real_qual/images/3_production_id_4122569_2160p'

In [23]:
for video_name in ["3_production_id_4122569_2160p"]: #image_dirs[::-1]:
    n_i = len(glob.glob(os.path.join(data_dir, "masks", video_name,"*/*.png")))
    n_f = len(os.listdir(os.path.join(data_dir, "images", video_name)))
    n_i = n_i // n_f
    !rm -rf {os.path.join(data_dir, "temp")}
    os.makedirs(os.path.join(data_dir, "temp"), exist_ok=True)
    
    
    prev_pred = []
    # try:
    all_names = sorted(os.listdir(os.path.join(data_dir, "images", video_name)))
    # all_names = []
    for inst_i in range(n_i):
        for image_name in all_names:
            image_name = image_name.replace(".jpg", "")
            all_paths = []
            for dirname in directories:
                if dirname == "images":
                    all_paths.append(os.path.join(data_dir, dirname, video_name, f"{image_name}.jpg"))
                else:
                    mask_path = os.path.join(data_dir, dirname, video_name, image_name, "{:02d}.png".format(inst_i))
                    if os.path.exists(mask_path):
                        all_paths.append(mask_path)
                    else:
                        all_paths.append(os.path.join(data_dir, dirname, video_name, image_name + ".png"))
            all_images = []
            for path in all_paths:
                all_images.append(np.array(Image.open(path).convert("RGB")))
            
            # Combine to single image
            all_images = all_images[:2] + [np.zeros_like(all_images[0])] + all_images[2:]
            row1 = np.concatenate(all_images[:3], axis=1)
            # row2 = np.concatenate(all_images[3:5], axis=1)

            # compose image with green background
            green_bg = np.zeros_like(all_images[0])
            green_bg[:, :, 1] = 255
            alpha = all_images[3] / 255.0
            image1 = all_images[0] * alpha + (1.0 - alpha) * green_bg
            alpha = all_images[4] / 255.0
            image2 = all_images[0] * alpha + (1.0 - alpha) * green_bg
            alpha = all_images[5] / 255.0
            image3 = all_images[0] * alpha + (1.0 - alpha) * green_bg
            row2 = np.concatenate([image1, image2, image3], axis=1)

            # green_bg = np.zeros_like(all_images[0])
            # green_bg[:, :, 1] = 255
            # alpha = all_images[5] / 255.0
            # image1 = all_images[0] * alpha + (1.0 - alpha) * green_bg
            # alpha = all_images[6] / 255.0
            # image2 = all_images[0] * alpha + (1.0 - alpha) * green_bg
            row3 = np.concatenate(all_images[3:], axis=1)

            combined = np.concatenate([row1, row2, row3], axis=0)
            # prev_pred = all_images[5:10]
            ratio = 2500.0 / combined.shape[1]
            cv2.putText(combined, image_name, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 5)
            combined = cv2.resize(combined, (0, 0), fx=ratio, fy=ratio)
            
            cv2.imwrite(os.path.join(data_dir, "temp", f"{image_name}.jpg"), combined[:, :, ::-1])
            
        out_vid = f"{data_dir}/{video_name}_{inst_i}.mp4"
        in_pattern = f"{data_dir}/temp/*.jpg"
        
        !ffmpeg -y -framerate 3 -pattern_type glob -i "{in_pattern}" -c:v libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p {out_vid}
    

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


In [17]:
video_name

'3_production_id_4122569_2160p'

In [8]:
row3.shape

(1012, 44160, 3)

In [3]:
directories = [
    "images_medium",
    "masks_medium",
    "alphas_medium",
    "comp_medium_none",
    "comp_medium_mono",
    "comp_medium_bi",
    "comp_medium_bi_fusion_mono",
    "comp_medium",
]
data_dir = "/home/chuongh/vm2m/output/VHM/ours_vhm_bi-temp_1108_2/vis_fusion_13k_removenoise"
image_dirs = sorted(os.listdir(os.path.join(data_dir, "images_medium")))

In [17]:
def sad(pred, gt):
    pred = pred[:, :, 0] / 255.0
    gt = gt[:, :, 0] / 255.0
    diff = np.abs(pred - gt)
    return np.sum(diff)

def dtssd(pred1, pred2, gt1, gt2):
    pred1 = pred1[:, :, 0] / 255.0
    pred2 = pred2[:, :, 0] / 255.0
    gt1 = gt1[:, :, 0] / 255.0
    gt2 = gt2[:, :, 0] / 255.0
    diff = ((pred1 - pred2) - (gt1 - gt2))**2
    return np.sum(diff)

results = []
for video_name in image_dirs:
    n_i = len(glob.glob(os.path.join(data_dir, "masks_medium", video_name,"*/*.png")))
    n_f = len(os.listdir(os.path.join(data_dir, "images_medium", video_name)))
    n_i = n_i // n_f
    !rm -rf {os.path.join(data_dir, "temp")}
    os.makedirs(os.path.join(data_dir, "temp"), exist_ok=True)
    
    for inst_i in range(n_i):
        prev_pred = []
        # try:
        all_names = sorted(os.listdir(os.path.join(data_dir, "masks_medium", video_name)))
        for image_i, image_name in enumerate(all_names):
            image_name = image_name.replace(".jpg", "")
            all_paths = []
            for dirname in directories:
                if dirname == "images_medium":
                    all_paths.append(os.path.join(data_dir, dirname, video_name, f"{image_name}.jpg"))
                else:
                    mask_path = os.path.join(data_dir, dirname, video_name, image_name, "{:02d}.png".format(inst_i))
                    if os.path.exists(mask_path):
                        all_paths.append(mask_path)
                    else:
                        all_paths.append(os.path.join(data_dir, dirname, video_name, image_name + ".png"))
            all_images = []
            for path in all_paths:
                all_images.append(np.array(Image.open(path).convert("RGB")))
            
            # Combine to single image
            row1 = all_images[:3] + [np.zeros_like(all_images[0])] * 2
            row1 = np.concatenate(row1, axis=1) # images, alphas, masks
            
            row2 = np.concatenate(all_images[3:], axis=1) # comp_none, comp_mono, comp_bi, comp_bi_fusion, comp

            diff1 = log_diff_cmap(prev_pred[0], all_images[3]) if len(prev_pred) > 0 else np.zeros_like(all_images[3])
            diff2 = log_diff_cmap(prev_pred[1], all_images[4]) if len(prev_pred) > 0 else np.zeros_like(all_images[4])
            diff3 = log_diff_cmap(prev_pred[2], all_images[5]) if len(prev_pred) > 0 else np.zeros_like(all_images[5])
            diff4 = log_diff_cmap(prev_pred[3], all_images[6]) if len(prev_pred) > 0 else np.zeros_like(all_images[6])
            diff5 = log_diff_cmap(prev_pred[4], all_images[7]) if len(prev_pred) > 0 else np.zeros_like(all_images[7])
            row3 = np.concatenate([diff1, diff2, diff3, diff4, diff5], axis=1)

            combined = np.concatenate([row1, row2, row3], axis=0)
            # Check target conidtion and printout the video_name, inst_i, image_name
            if len(prev_pred) > 0:
                sad0 = sad(all_images[3], all_images[1])
                sad1 = sad(all_images[4], all_images[1])
                sad2 = sad(all_images[5], all_images[1])
                sad3 = sad(all_images[6], all_images[1])
                sad4 = sad(all_images[7], all_images[1])
                dtssd0 = dtssd(all_images[3], prev_pred[0], all_images[1], prev_pred[-1])
                dtssd1 = dtssd(all_images[4], prev_pred[1], all_images[1], prev_pred[-1])
                dtssd2 = dtssd(all_images[5], prev_pred[2], all_images[1], prev_pred[-1])
                dtssd3 = dtssd(all_images[6], prev_pred[3], all_images[1], prev_pred[-1])
                dtssd4 = dtssd(all_images[7], prev_pred[4], all_images[1], prev_pred[-1])
                if dtssd4 < min(dtssd0, dtssd1, dtssd2, dtssd3) and dtssd0 > max(dtssd1, dtssd2) and sad4 < min(sad0, sad1, sad2, sad3) and sad0 > max(sad1, sad2):
                    print(video_name, inst_i, all_names[image_i - 1])
                    results.append([video_name, inst_i, all_names[image_i - 1]])
            prev_pred = all_images[3:] + [all_images[1]]
            ratio = 2500.0 / combined.shape[1]
            combined = cv2.resize(combined, (0, 0), fx=ratio, fy=ratio)
            cv2.putText(combined, image_name, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 5)

            cv2.imwrite(os.path.join(data_dir, "temp", f"{image_name}.jpg"), combined[:, :, ::-1])

        out_vid = f"{data_dir}/1126_{video_name}_{inst_i}.mp4"
        in_pattern = f"{data_dir}/temp/*.jpg"
        !ffmpeg -y -framerate 3 -pattern_type glob -i "{in_pattern}" -c:v libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p {out_vid}

00029 0 00012
00029 0 00014
00029 0 00027
ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100

In [15]:
ratio

0.2604166666666667

In [8]:
results

[['00029', 0, '00012'],
 ['00029', 0, '00014'],
 ['00029', 0, '00027'],
 ['00029', 2, '00021'],
 ['00033', 1, '00011'],
 ['00037', 0, '00022'],
 ['00050', 0, '00020'],
 ['00050', 1, '00015'],
 ['00058', 0, '00014'],
 ['00058', 0, '00020'],
 ['00058', 0, '00023'],
 ['00058', 1, '00028'],
 ['00064', 1, '00019'],
 ['00066', 1, '00011'],
 ['00080', 0, '00011'],
 ['00102', 0, '00025'],
 ['00102', 0, '00027'],
 ['00102', 1, '00028'],
 ['00112', 0, '00004'],
 ['00112', 0, '00018'],
 ['00127', 0, '00011'],
 ['00127', 0, '00017'],
 ['00127', 1, '00014'],
 ['00157', 0, '00007'],
 ['00157', 0, '00027'],
 ['00168', 1, '00006'],
 ['00168', 2, '00015'],
 ['00168', 2, '00017'],
 ['00168', 2, '00018'],
 ['00221', 0, '00020'],
 ['00221', 0, '00022'],
 ['00221', 0, '00023'],
 ['00221', 0, '00024'],
 ['00221', 0, '00025'],
 ['00221', 1, '00017']]